# Manipulation methods

- We use them to perform operations on pandas Series
- In this chapter it's explored the `.apply` and `.where` methods

In [23]:
import pandas as pd
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url)
city_mpg = df.city08

/home/luis/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Applying functions to pandas Series

`.apply` allows us to apply some function to a pandas Series. It comes in handy many times, but is that more performatic? And, is it also indiomatic pandas? Let's see that.

First, we create a function to compare if a mileage value is greater than 20 and compare the time execution with the `gt` method

In [2]:
def gt20(val):
    return val > 20

In [3]:
%%timeit
city_mpg.apply(gt20)

7.87 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
city_mpg.gt(20)

377 µs ± 69.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


It's incredible the broadcasted method `gt` is ~123x faster than the apply method 🤯. Well, the `apply` method will broadcast the operation, and the operation will be called for every value.

Let's see another example. The below example shows us the make column, which contains the name of company that made each car:

In [24]:
make = df.make
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

Now I might want to limit my dataset to show the top five makes and label everything else as Other. Using the `value_counts` method it's easy to see the top 5 makes:

In [6]:
make.value_counts()

Chevrolet                           4003
Ford                                3371
Dodge                               2583
GMC                                 2494
Toyota                              2071
                                    ... 
Panoz Auto-Development                 1
Koenigsegg                             1
Environmental Rsch and Devp Corp       1
Excalibur Autos                        1
Shelby                                 1
Name: make, Length: 136, dtype: int64

Again, it will be compared the performance of `apply` and a broadcasted method. Separate the top 5 makers and create the function

In [7]:
top5 = make.value_counts()[:5]

def generalize_top5(val):
    if val in top5:
        return val
    return 'Other'

Observe the dtype of the `top5` variable! It will be important in the future

In [8]:
type(top5)

pandas.core.series.Series

In [9]:
make.apply(generalize_top5)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

The method we'll use is the `where` method which replace the values where some condition is false. To make the comparison, it will be used the `isin` method. Basically tt takes a *boolean array* and compare the values with some value.

**Remember**: The `Series.isin` iterates over *something*, where something is a list. To make the correct comparison, it must get the index. [See here more information](https://stackoverflow.com/questions/32978362/problems-with-isin-pandas)

In [10]:
make.isin(top5.index)

0        False
1        False
2         True
3         True
4        False
         ...  
41139    False
41140    False
41141    False
41142    False
41143    False
Name: make, Length: 41144, dtype: bool

As it's seen, the same result is achieved

In [11]:
make.where(
    make.isin(top5.index),
    other='Other'
)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

In [12]:
%%timeit
make.apply(generalize_top5)

39.8 ms ± 253 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
make.where(make.isin(top5.index), other='Other')

2.86 ms ± 52.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Compare the times, again it's possible to see the `apply` is not so performatic!

As bonus, here is the complement of `where` is the `mask` method. It does the same thing, but instead of replacing the False values, it will replace the True values

In [14]:
make.mask(~make.isin(top5.index), other='Other')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

## 9.2 if Else with Pandas

Imagine this: 
- I want to keep in mind the top 5 and top 10 values, we create this function

In [15]:
vc = make.value_counts()
top5 = vc.index[:5]
top10 = vc.index[:10]

In [16]:
def generalize(val):
    if val in top5:
        return val
    elif val in top10:
        return 'Top10'
    else:
        return 'Other'

In [17]:
make.apply(generalize)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

To replicate the same result in pandas, we can use chain calls to `.where`

In [18]:
(make
    .where(make.isin(top5), 'Top10')
    .where(make.isin(top10), 'Other')
)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: object

We can also use numpy to perform the same operation

In [19]:
import numpy as np

np.select([make.isin(top5), make.isin(top10)], [make, 'Top10'], 'Other')

array(['Other', 'Other', 'Dodge', ..., 'Other', 'Other', 'Other'],
      dtype=object)

Here we have a problem: the numpy method will return a numpy array. However, we can wrap this data around a pandas Series

In [20]:
pd.Series(np.select([make.isin(top5), make.isin(top10)], [make, 'Top10'], 'Other'), index=make.index)

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Length: 41144, dtype: object

## 9.3 Missing Data

Deal with missing data is a common thing in the data analist/engineer. The data in the real world is not perfect, and it's always missing something. The cylinder column has missing data, so let's use this column to perform some operations and learn how to deal with the missing data

In [21]:
cyl = df.cylinders

In [22]:
cyl

0         4.0
1        12.0
2         4.0
3         8.0
4         4.0
         ... 
41139     4.0
41140     4.0
41141     4.0
41142     4.0
41143     4.0
Name: cylinders, Length: 41144, dtype: float64

In [23]:
(cyl
    .isna()
    .sum()
)

206

So there are 206 fields missing data

In [24]:
missing = cyl.isna()

In [25]:
make.loc[missing]

7138     Nissan
7139     Toyota
8143     Toyota
8144       Ford
8146       Ford
          ...  
34563     Tesla
34564     Tesla
34565     Tesla
34566     Tesla
34567     Tesla
Name: make, Length: 206, dtype: object

How do we deal with the missing data? In the next sub chapters let's explore different solve this problem and some others methods, like sort and rank data

## 9.4 Filling in Missing Data

The first approach is filling the missing data with some value. It's possible to do this by using the `fillna` method (it doesn't change the data itself)

In [26]:
cyl[cyl.isna()]

7138    NaN
7139    NaN
8143    NaN
8144    NaN
8146    NaN
         ..
34563   NaN
34564   NaN
34565   NaN
34566   NaN
34567   NaN
Name: cylinders, Length: 206, dtype: float64

In [27]:
cyl.fillna(0).loc[7136:7141]

7136    6.0
7137    6.0
7138    0.0
7139    0.0
7140    6.0
7141    6.0
Name: cylinders, dtype: float64

## 9.5 Interpolate data

- Operation that determines a value from the previous and next values 
- It does work when data is sorted

In [28]:
temp = pd.Series([32, 40, None, 42, 39, 32])
temp

0    32.0
1    40.0
2     NaN
3    42.0
4    39.0
5    32.0
dtype: float64

In [29]:
temp.interpolate()

0    32.0
1    40.0
2    41.0
3    42.0
4    39.0
5    32.0
dtype: float64

In [34]:
test = pd.Series([0, 10, 18, 12, None, 7, 8], dtype=float)

In [35]:
test

0     0.0
1    10.0
2    18.0
3    12.0
4     NaN
5     7.0
6     8.0
dtype: float64

In [36]:
test.interpolate()

0     0.0
1    10.0
2    18.0
3    12.0
4     9.5
5     7.0
6     8.0
dtype: float64

## 9.6 Clipping Data

In [37]:
city_mpg.loc[:446]

0      19
1       9
2      23
3      10
4      17
       ..
442    15
443    15
444    15
445    15
446    31
Name: city08, Length: 447, dtype: int64

In [38]:
(city_mpg
    .loc[:446]
    .clip(lower=city_mpg.quantile(.05),
         upper=city_mpg.quantile(.95))
)

0      19
1      11
2      23
3      11
4      17
       ..
442    15
443    15
444    15
445    15
446    27
Name: city08, Length: 447, dtype: int64

## 9.8 Sorting values

In [39]:
city_mpg.sort_values()

7901       6
34557      6
37161      6
21060      6
35887      6
        ... 
34563    138
34564    140
32599    150
31256    150
33423    150
Name: city08, Length: 41144, dtype: int64

## 9.9 Sorting index

In [42]:
city_mpg.sort_values().sort_index()

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: int64

## 9.10 Ranking Data

- The `.rank` method will return a series that keeps the original index but uses the ranks of values from the original series.
- The `.rank` function is used to compute numerical data ranks (1 through n) along axis. By default, equal values are assigned a rank that is the average of the ranks of those values.

In [43]:
city_mpg.rank()

0        27060.5
1          235.5
2        35830.0
3          607.5
4        19484.0
          ...   
41139    27060.5
41140    29719.5
41141    23528.0
41142    23528.0
41143    15479.0
Name: city08, Length: 41144, dtype: float64

- 'min' parameter will put equal values in the same rank
- 'dense'  parameter won't skip any positions

In [44]:
city_mpg.rank(method='min')

0        25555.0
1          136.0
2        35119.0
3          336.0
4        17467.0
          ...   
41139    25555.0
41140    28567.0
41141    21502.0
41142    21502.0
41143    13492.0
Name: city08, Length: 41144, dtype: float64

In [45]:
city_mpg.rank(method='dense')

0        14.0
1         4.0
2        18.0
3         5.0
4        12.0
         ... 
41139    14.0
41140    15.0
41141    13.0
41142    13.0
41143    11.0
Name: city08, Length: 41144, dtype: float64

## 9.11 Replacing Data

- Useful when you want to change certain values from the data frame/series

In [47]:
make.replace('Subaru', 'スバル')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4               スバル
            ...    
41139           スバル
41140           スバル
41141           スバル
41142           スバル
41143           スバル
Name: make, Length: 41144, dtype: object

It's possible to use regex!

In [48]:
make.replace(r'(Fer)ra(r.*)', value=r'\2-other-\1', regex=True)

0          Alfa Romeo
1        ri-other-Fer
2               Dodge
3               Dodge
4              Subaru
             ...     
41139          Subaru
41140          Subaru
41141          Subaru
41142          Subaru
41143          Subaru
Name: make, Length: 41144, dtype: object

## 9.12 Binning data

In [49]:
pd.cut(city_mpg, 10)

0        (5.856, 20.4]
1        (5.856, 20.4]
2         (20.4, 34.8]
3        (5.856, 20.4]
4        (5.856, 20.4]
             ...      
41139    (5.856, 20.4]
41140    (5.856, 20.4]
41141    (5.856, 20.4]
41142    (5.856, 20.4]
41143    (5.856, 20.4]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64]): [(5.856, 20.4] < (20.4, 34.8] < (34.8, 49.2] < (49.2, 63.6] ... (92.4, 106.8] < (106.8, 121.2] < (121.2, 135.6] < (135.6, 150.0]]

### What does this result mean?

Basically it means we have 10 values in each of that categories. Notice the dtype was changed to categorical

We can also specify the bins, passing a list of values

In [50]:
pd.cut(city_mpg, [0 , 10 , 20 , 40 , 70 , 150])

0        (10, 20]
1         (0, 10]
2        (20, 40]
3         (0, 10]
4        (10, 20]
           ...   
41139    (10, 20]
41140    (10, 20]
41141    (10, 20]
41142    (10, 20]
41143    (10, 20]
Name: city08, Length: 41144, dtype: category
Categories (5, interval[int64]): [(0, 10] < (10, 20] < (20, 40] < (40, 70] < (70, 150]]

Bin data with quatiles:

In [51]:
pd.qcut(city_mpg, 10)

0         (18.0, 20.0]
1        (5.999, 13.0]
2         (21.0, 24.0]
3        (5.999, 13.0]
4         (16.0, 17.0]
             ...      
41139     (18.0, 20.0]
41140     (18.0, 20.0]
41141     (17.0, 18.0]
41142     (17.0, 18.0]
41143     (15.0, 16.0]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64]): [(5.999, 13.0] < (13.0, 14.0] < (14.0, 15.0] < (15.0, 16.0] ... (18.0, 20.0] < (20.0, 21.0] < (21.0, 24.0] < (24.0, 150.0]]

# Exercises

1. Create a series from a numeric column that has the value of 'high' if it is equal to or above the mean and 'low' if it is below the mean using `.apply`.

In [1]:
import pandas as pd

In [3]:
heights = pd.Series([1.85, 1.83, 1.70, 2.01, 1.64, 1.35, 1.16, 1.20, 1.45, 1.70, 1.82])
heights

0     1.85
1     1.83
2     1.70
3     2.01
4     1.64
5     1.35
6     1.16
7     1.20
8     1.45
9     1.70
10    1.82
dtype: float64

In [4]:
mean = heights.mean()

def classify_high(height):
    if height >= mean:
        return 'high'
    return 'low'

In [5]:
heights.apply(classify_high)

0     high
1     high
2     high
3     high
4     high
5      low
6      low
7      low
8      low
9     high
10    high
dtype: object

2. Create a series from a numeric column that has the value of 'high' if it is equal to or above the mean and 'low' if it is below the mean using `np.select`.

In [6]:
heights

0     1.85
1     1.83
2     1.70
3     2.01
4     1.64
5     1.35
6     1.16
7     1.20
8     1.45
9     1.70
10    1.82
dtype: float64

In [8]:
import numpy as np

mean = heights.mean()
np.select([heights >= mean, heights < mean], ['high', 'low'])

array(['high', 'high', 'high', 'high', 'high', 'low', 'low', 'low', 'low',
       'high', 'high'], dtype='<U21')

3. Time the differences between the previous two solutions to see which is faster.

In [9]:
%%timeit
heights.apply(classify_high)

270 µs ± 32.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
np.select([heights >= mean, heights < mean], ['high', 'low'])

450 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


4. Replace the missing values of a numeric series with the median value.

In [12]:
heights = pd.Series([1.85, 1.83, None, 1.70, 2.01, 1.64, None, 1.35, 1.16, 1.20, None, 1.45, 1.70, 1.82, None])
heights

0     1.85
1     1.83
2      NaN
3     1.70
4     2.01
5     1.64
6      NaN
7     1.35
8     1.16
9     1.20
10     NaN
11    1.45
12    1.70
13    1.82
14     NaN
dtype: float64

In [14]:
mean = heights.mean()
mean

1.6099999999999997

In [15]:
heights.fillna(mean)

0     1.85
1     1.83
2     1.61
3     1.70
4     2.01
5     1.64
6     1.61
7     1.35
8     1.16
9     1.20
10    1.61
11    1.45
12    1.70
13    1.82
14    1.61
dtype: float64

5. Clip the values of a numeric series to between to 10th and 90th percentiles.

In [19]:
heights = pd.Series([1.85, 1.83, 1.70, 2.01, 1.64, 1.35, 1.16, 1.20, 1.45, 1.70, 1.82])
heights

0     1.85
1     1.83
2     1.70
3     2.01
4     1.64
5     1.35
6     1.16
7     1.20
8     1.45
9     1.70
10    1.82
dtype: float64

In [21]:
heights.clip(lower=heights.quantile(.10), upper=heights.quantile(.90))

0     1.85
1     1.83
2     1.70
3     1.85
4     1.64
5     1.35
6     1.20
7     1.20
8     1.45
9     1.70
10    1.82
dtype: float64

6. Using a categorical column, replace any value that is not in the top 5 most frequent values with 'Other'.

In [42]:
# Loading data
make = df.make
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [43]:
# Getting top 5
top5 = make.value_counts()[:5]
top5

Chevrolet    4003
Ford         3371
Dodge        2583
GMC          2494
Toyota       2071
Name: make, dtype: int64

In [46]:
make.where(
    make.isin(top5.index),
    other='Other'
).astype('category')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: category
Categories (6, object): ['Chevrolet', 'Dodge', 'Ford', 'GMC', 'Other', 'Toyota']

7. Using a categorical column, replace any value that is not in the top 10 most frequent values with 'Other'.

In [47]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [49]:
top10 = make.value_counts()[:10]

make.where(
    make.isin(top10.index),
    other='Other'
).astype('category')

0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: category
Categories (11, object): ['BMW', 'Chevrolet', 'Dodge', 'Ford', ..., 'Nissan', 'Other', 'Toyota', 'Volkswagen']

8. Make a function that takes a categorical series and a number (n) and returns a replace series that replaces any value that is not in the top n most frequent values with 'Other'.

In [51]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [55]:
n = int(input('Insert top n: '))

topN = make.value_counts()[:n]

make.where(
    make.isin(topN.index),
    other='Other'
).astype('category')

Insert top n:  15


0        Other
1        Other
2        Dodge
3        Dodge
4        Other
         ...  
41139    Other
41140    Other
41141    Other
41142    Other
41143    Other
Name: make, Length: 41144, dtype: category
Categories (16, object): ['Audi', 'BMW', 'Chevrolet', 'Dodge', ..., 'Other', 'Porsche', 'Toyota', 'Volkswagen']

9. Using a numeric column, bin it into 10 groups that have the same width.

In [56]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url)
city_mpg = df.city08

/home/luis/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
pd.cut(city_mpg, 10)

0        (5.856, 20.4]
1        (5.856, 20.4]
2         (20.4, 34.8]
3        (5.856, 20.4]
4        (5.856, 20.4]
             ...      
41139    (5.856, 20.4]
41140    (5.856, 20.4]
41141    (5.856, 20.4]
41142    (5.856, 20.4]
41143    (5.856, 20.4]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64]): [(5.856, 20.4] < (20.4, 34.8] < (34.8, 49.2] < (49.2, 63.6] ... (92.4, 106.8] < (106.8, 121.2] < (121.2, 135.6] < (135.6, 150.0]]

10. Using a numeric column, bin it into 10 groups that have equal sized bins.

In [58]:
pd.qcut(city_mpg, 10)

0         (18.0, 20.0]
1        (5.999, 13.0]
2         (21.0, 24.0]
3        (5.999, 13.0]
4         (16.0, 17.0]
             ...      
41139     (18.0, 20.0]
41140     (18.0, 20.0]
41141     (17.0, 18.0]
41142     (17.0, 18.0]
41143     (15.0, 16.0]
Name: city08, Length: 41144, dtype: category
Categories (10, interval[float64]): [(5.999, 13.0] < (13.0, 14.0] < (14.0, 15.0] < (15.0, 16.0] ... (18.0, 20.0] < (20.0, 21.0] < (21.0, 24.0] < (24.0, 150.0]]